# Pathway analysis of metagenomic data

First, install the libraries and import them


In [ ]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("clusterProfiler")
BiocManager::install("org.Hs.eg.db")
BiocManager::install("org.EcK12.eg.db")
BiocManager::install("KEGGREST")
BiocManager::install("limma")

In [ ]:
library(clusterProfiler)
library(KEGGREST)
#library(org.Hs.eg.db)
library(org.EcK12.eg.db)
library(limma)
#keytypes(org.EcK12.eg.db)

In [1]:
# Import the metagenomics data
metagenomics <- read.csv(file = '../Data/ecs_relab_split.csv', sep = '\t', header = TRUE)
head(metagenomics)

,Gene.Family,Name.Organism,CSM5FZ4M,CSM5MCUO,CSM5MCVL,CSM5MCVN,CSM5MCW6,CSM5MCWC,CSM5MCWE,CSM5MCWG,...,CSM5MCVJ_P,CSM5MCWI_P,MSM5LLHA_P,HSM5MD59_P,HSM5MD8N_P,CSM6J2H9_P,HSM5MD4A_P,CSM5MCUW_P,ESM5MEBA_P,HSM5FZC2_P
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase,0.00183508,0.00147242,0.00157135,0.00117332,0.000246863,0.00238904,0.000922349,0.000573488,...,0.00172861,0.00180843,0.000636998,0.00190471,0.00168832,0.00183791,0.00191878,0.00139149,0.00124719,0.000429274
2,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Aggregatibacter.s__Aggregatibacter_segnis,0,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000,0.000000000,0.000000000,...,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000
3,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Alloprevotella.s__Alloprevotella_tannerae,0,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000,0.000000000,0.000000000,...,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000
4,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Anaerococcus.s__Anaerococcus_obesiensis,0,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000,0.000000000,0.000000000,...,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000
5,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Anaerococcus.s__Anaerococcus_vaginalis,0,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000,0.000000000,0.000000000,...,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000
6,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Anaerofustis.s__Anaerofustis_stercorihominis,0,0.00000000,0.00000000,0.00000000,0.000000000,0.00000000,0.000000000,0.000000000,...,0.00000000,0.00000000,0.000000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.00000000,0.000000000


In [3]:
# Subset Escherichia coli
metagenomics_Ecoli <- metagenomics[grepl("Escherichia_coli", metagenomics$Name.Organism, fixed = TRUE),]
head(metagenomics_Ecoli)

,Gene.Family,Name.Organism,CSM5FZ4M,CSM5MCUO,CSM5MCVL,CSM5MCVN,CSM5MCW6,CSM5MCWC,CSM5MCWE,CSM5MCWG,...,CSM5MCVJ_P,CSM5MCWI_P,MSM5LLHA_P,HSM5MD59_P,HSM5MD8N_P,CSM6J2H9_P,HSM5MD4A_P,CSM5MCUW_P,ESM5MEBA_P,HSM5FZC2_P
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
72,1.1.1.100,3-oxoacyl-[acyl-carrier-protein] reductase|g__Escherichia.s__Escherichia_coli,4.1268e-06,9.91858e-06,5.74953e-06,0.0000e+00,0,3.55339e-05,0.000111472,0.000305886,...,2.89118e-05,2.52982e-05,5.73950e-05,3.89813e-06,0.0000e+00,0,4.10781e-06,2.61897e-04,0,4.94513e-06
209,1.1.1.103,L-threonine 3-dehydrogenase|g__Escherichia.s__Escherichia_coli,4.10204e-06,5.64941e-06,7.84576e-06,0.0000e+00,0,7.52441e-05,0.000275560,0.000665927,...,4.38022e-06,1.80096e-05,1.18570e-05,9.38069e-06,1.5674e-06,0,2.81095e-06,9.14451e-05,0,4.35009e-06
250,1.1.1.11,D-arabinitol 4-dehydrogenase|g__Escherichia.s__Escherichia_coli,0,0.00000e+00,0.00000e+00,0.0000e+00,0,0.00000e+00,0.000000000,0.000000000,...,0.00000e+00,0.00000e+00,0.00000e+00,0.00000e+00,0.0000e+00,0,0.00000e+00,0.00000e+00,0,0.00000e+00
288,1.1.1.127,2-dehydro-3-deoxy-D-gluconate 5-dehydrogenase|g__Escherichia.s__Escherichia_coli,0,6.37618e-07,9.84720e-06,1.8641e-06,0,1.36083e-04,0.000637147,0.001320120,...,2.43733e-05,4.50702e-05,2.57285e-05,1.16513e-05,0.0000e+00,0,1.88847e-05,1.64357e-04,0,3.02259e-06
306,1.1.1.130,3-dehydro-L-gulonate 2-dehydrogenase|g__Escherichia.s__Escherichia_coli,1.05122e-06,4.19287e-06,9.47301e-06,2.8743e-06,0,7.07101e-05,0.000361889,0.000844426,...,2.53331e-05,2.87693e-05,3.11684e-05,8.45031e-06,0.0000e+00,0,4.34527e-06,2.28527e-04,0,5.24028e-06
356,1.1.1.133,dTDP-4-dehydrorhamnose reductase|g__Escherichia.s__Escherichia_coli,0,0.00000e+00,0.00000e+00,0.0000e+00,0,3.03047e-05,0.000133478,0.000215957,...,0.00000e+00,0.00000e+00,0.00000e+00,6.65494e-07,0.0000e+00,0,0.00000e+00,2.66236e-04,0,9.27221e-06


# Differential gene expression analysis

In [4]:
# Import metadata file
metadata <- read.csv(file = '../Data/hmp2_metadata_2018-08-20.csv', sep = ',', header = TRUE)
# Select metagenomics
metagenomics_metadata <- metadata[metadata$data_type=='metagenomics',]
# Write metagenomics metadata file into csv
write.csv(metagenomics_metadata,"..\\Data\\Metagenomics_metadata.csv", row.names = FALSE)

In [5]:
metagenomics_metadata <- read.csv(file = '../Data/Metagenomics_metadata.csv', sep = ',', header = TRUE)
metagenomics_metadata_subset <- subset(metagenomics_metadata, select= c(External.ID, diagnosis, visit_num))
metagenomics_metadata_subset <- metagenomics_metadata_subset[metagenomics_metadata_subset$visit_num==4,]
head(metagenomics_metadata_subset)

,External.ID,diagnosis,visit_num
,<chr>,<chr>,<int>
1,CSM5FZ3N_P,CD,4
17,CSM5FZ3T_P,CD,4
42,CSM5FZ4A_P,UC,4
78,CSM5MCTZ_P,UC,4
89,CSM5MCVB_P,CD,4
92,CSM5MCU4_P,CD,4


In [6]:
# Separate CD, UC and nonIBD
cd <- metagenomics_metadata_subset[metagenomics_metadata_subset$diagnosis=="CD",]
uc <- metagenomics_metadata_subset[metagenomics_metadata_subset$diagnosis=="UC",]
nonIBD <- metagenomics_metadata_subset[metagenomics_metadata_subset$diagnosis=="nonIBD",]

In [7]:
# select metagenomics data from CD, UC and nonIBD
metagenomics_cd <- subset(metagenomics_Ecoli, select= c("Gene.Family", cd$External.ID))
metagenomics_uc <- subset(metagenomics_Ecoli, select= c("Gene.Family", uc$External.ID))
metagenomics_nonIBD <- subset(metagenomics_Ecoli, select= c("Gene.Family", nonIBD$External.ID))

In [8]:
head(metagenomics_cd)

,Gene.Family,CSM5FZ3N_P,CSM5FZ3T_P,CSM5MCVB_P,CSM5MCU4_P,CSM5MCWK_P,CSM5MCXF_P,CSM67U9V_P,CSM67U9X_P,CSM67UAI_P,...,MSM9VZLX_P,MSM9VZM4_P,PSM6XBQM_P,PSM6XBRK_P,PSM6XBRK,PSM6XBRK_TR,PSM6XBRM_P,PSM6XBVY_P,PSM7J1B3_P,PSM7J4EF
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
72,1.1.1.100,0,0,2.90917e-04,0,0,1.53931e-05,1.00766e-05,4.61144e-06,0.000209820,...,0,0,0,0,0,0,1.34360e-06,0,0.000444394,1.45595e-04
209,1.1.1.103,0,0,1.17246e-04,0,0,1.93987e-06,4.34007e-06,2.70102e-06,0.000318774,...,0,0,0,0,0,0,1.00422e-06,0,0.000152316,9.05002e-05
250,1.1.1.11,0,0,0.00000e+00,0,0,0.00000e+00,0.00000e+00,0.00000e+00,0.000000000,...,0,0,0,0,0,0,0.00000e+00,0,0.000000000,0.00000e+00
288,1.1.1.127,0,0,1.98709e-04,0,0,2.17053e-05,6.29250e-05,1.90161e-06,0.000602487,...,0,0,0,0,0,0,0.00000e+00,0,0.000291561,8.23561e-05
306,1.1.1.130,0,0,3.40738e-04,0,0,3.59047e-05,2.12171e-05,5.60158e-06,0.000510456,...,0,0,0,0,0,0,4.08567e-06,0,0.000408950,7.35535e-05
356,1.1.1.133,0,0,4.10046e-06,0,0,8.99820e-06,0.00000e+00,0.00000e+00,0.000269802,...,0,0,0,0,0,0,6.95513e-06,0,0.000251756,1.32574e-04


In [9]:
# Compute statistical significance (using t-test)
pvalue_cd = NULL # Empty list for the p-values
tstat_cd = NULL # Empty list of the t test statistics

for(i in 1 : nrow(metagenomics_Ecoli)) { # For each gene : 
	x = metagenomics_nonIBD[i,-1] # control of gene number i
	y = metagenomics_cd[i,-1] # CD of gene number i
	
	# Compute t-test between the two conditions
	t = t.test(x, y)
	
	# Put the current p-value in the pvalues list
	pvalue_cd[i] = t$p.value
	# Put the current t-statistic in the tstats list
	tstat_cd[i] = t$statistic
}

In [10]:
# Compute statistical significance (using t-test)
subset_NA = NULL

for(i in 1 : nrow(metagenomics_Ecoli)) { # For each gene : 

	x = metagenomics_nonIBD[i,-1] # Control of gene number i
	y = metagenomics_uc[i,-1] # UC of gene number i

	metagenomics_nonIBD_NA = all(is.na(x)) || length(is.na(x)==FALSE)<2
	metagenomics_uc_NA = all(is.na(y)) || length(is.na(y)==FALSE)<2
	subset_NA[i] = metagenomics_nonIBD_NA || metagenomics_uc_NA
	
}
metagenomics_uc_nonNA <- metagenomics_uc[!subset_NA,]
metagenomics_nonIBD_nonNA <- metagenomics_nonIBD[!subset_NA,]

In [11]:
# Compute statistical significance (using t-test)
pvalue_uc = NULL # Empty list for the p-values
tstat_uc = NULL # Empty list of the t test statistics

for(i in 1 : nrow(metagenomics_nonIBD_nonNA)) { # For each gene : 
	x = metagenomics_nonIBD_nonNA[i,-1] # control of gene number i
	y = metagenomics_uc_nonNA[i,-1] # UC of gene number i	
	
	# Compute t-test between the two conditions
	t = t.test(x, y)
	
	# Put the current p-value in the pvalues list
	pvalue_uc[i] = t$p.value
	# Put the current t-statistic in the tstats list
	tstat_uc[i] = t$statistic
}

In [12]:
metagenomics_Ecoli$pvalue_cd <- pvalue_cd
metagenomics_Ecoli$pvalue_uc <- pvalue_uc
cd_genes <- subset(metagenomics_Ecoli[metagenomics_Ecoli$pvalue_cd<=0.05,], select="Gene.Family", "")
uc_genes <- subset(metagenomics_Ecoli[metagenomics_Ecoli$pvalue_uc<=0.05,], select="Gene.Family")

In [42]:
#metagenomics_Ecoli[order(metagenomics_Ecoli$pvalue_cd)]
dim(metagenomics_Ecoli)

[1]  927 1642

In [49]:
#list of all deg from two disease types 
deg.CD  <- metagenomics_Ecoli[!is.na(metagenomics_Ecoli$pvalue_cd) & metagenomics_Ecoli$pvalue_cd < 0.05,"Gene.Family"]
#CD.up   <-unique(dataset.CD[!is.na(metagenomics_Ecoli$pvalue_cd) & metagenomics_Ecoli$pvalue_cd < 0.05,c(1,2)])
#CD.down <-unique(dataset.CD[!is.na(metagenomics_Ecoli$pvalue_cd) & metagenomics_Ecoli$pvalue_cd < 0.05,c(1,2)])

deg.UC  <- metagenomics_Ecoli[!is.na(metagenomics_Ecoli$pvalue_uc) & metagenomics_Ecoli$pvalue_uc < 0.05,"Gene.Family"]
#UC.up   <-unique(dataset.UC[!is.na(dataset.UC$pvalue) & dataset.UC$pvalue < 0.05,c(1,2)])
#UC.down <-unique(dataset.UC[!is.na(dataset.UC$pvalue) & dataset.UC$pvalue < 0.05,c(1,2)])


In [51]:
deg.CD

character(0)

## KEGG pathway over-representation analysis

Codes for KEGG organisms can be found here (https://www.genome.jp/kegg/catalog/org_list.html)

In [25]:
# Convert EC numbers to Entrez IDs
gene <- clusterProfiler::bitr(cd_genes$Gene.Family, fromType = "ENZYME", toType = "ENTREZID", OrgDb = org.EcK12.eg.db)

# Convert Entrez IDs to KEGG IDs
geneList <- sub("^", "ncbi-geneid:", gene[,2])
geneList <- keggConv("eco", geneList)

ERROR: Error in .testForValidKeys(x, keys, keytype, fks): None of the keys entered are valid keys for 'ENZYME'. Please use the keys method to see a listing of valid arguments.


In [20]:
# Remove preceding 'eco:'
geneList <- gsub("eco:", "", geneList)

kk <- enrichKEGG(gene         = geneList,
                 organism     = 'eco',
                 pvalueCutoff = 0.05)
head(kk)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'gsub': object 'geneList' not found


In [33]:
# Convert EC numbers to Entrez IDs
gene2 <- clusterProfiler::bitr(uc_genes$Gene.Family, fromType = "ENZYME", toType = "ENTREZID", OrgDb = org.EcK12.eg.db)

# Convert Entrez IDs to KEGG IDs
geneList2 <- sub("^", "ncbi-geneid:", gene2[,2])
geneList2 <- keggConv("eco", geneList2)

ERROR: Error in .testForValidKeys(x, keys, keytype, fks): None of the keys entered are valid keys for 'ENZYME'. Please use the keys method to see a listing of valid arguments.


In [34]:
# Remove preceding 'eco:'
geneList2 <- gsub("eco:", "", geneList2)

kk2 <- enrichKEGG(gene         = geneList2,
                 organism     = 'eco',
                 pvalueCutoff = 0.05)
head(kk2)

ERROR: Error in h(simpleError(msg, call)): error in evaluating the argument 'x' in selecting a method for function 'gsub': object 'geneList2' not found
